In [2]:
import os, sys, inspect
import torch
import numpy as np

data_path = "/home/cxl173430/data/uncertainty_Related/HENN_Git_VScode/HyperEvidentialNN/"
sys.path.insert(1, data_path)

from data.tinyImageNet import tinyImageNetVague
from data.cifar100 import CIFAR100Vague
from common_tools import set_device, create_path, dictToObj, set_random_seeds

set_random_seeds(42)
gpu = 0
device = set_device(gpu)

data_dir = '/home/cxl173430/data/DATASETS/'
batch_size = 64
num_comp = 10
gauss_kernel_size = 3


# mydata = CIFAR100Vague(
#                     data_dir, 
#                     num_comp=num_comp,
#                     batch_size=batch_size,
#                     duplicate=False,
#                     blur=True,
#                     gauss_kernel_size=gauss_kernel_size,
#                     pretrain=False,
#                     num_workers=4,
#                     seed=42,
#                     comp_el_size=2,
#                     )

mydata = tinyImageNetVague(
            data_dir, 
            num_comp=num_comp, 
            batch_size=batch_size,
            imagenet_hierarchy_path=data_dir,
            blur=True,
            gray=False,
            gauss_kernel_size=gauss_kernel_size,
            pretrain=True,
            num_workers=4,
            seed=42)


/home/cxl173430/.local/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Random Seed is set: 42
Using PyTorch version: 1.13.0
use gpu indexed: 0
Loading TinyImageNet...
length of parent_to_subclass:  107
Total 29 Candidate superclasses: ['n04371563', 'n03419014', 'n04490091', 'n02924116', 'n02858304', 'n04379243', 'n07557434', 'n07611358', 'n07707451', 'n03748162', 'n03259505', 'n02898711', 'n03743902', 'n02796623', 'n09433442', 'n03278248', 'n04438304', 'n04147495', 'n04565375', 'n03880531', 'n02876657', 'n01767661', 'n01772222', 'n01942177', 'n02470325', 'n02370806', 'n02075296', 'n02121808', 'n01844917']
Vague classes: ['n02876657', 'n02924116', 'n04371563', 'n01942177', 'n07707451', 'n07611358', 'n02370806', 'n02858304', 'n02121808', 'n04147495']
Vague classes nid: [['n02823428', 'n03983396'], ['n04146614', 'n04487081'], ['n02837789', 'n04371430'], ['n01944390', 'n01945685', 'n01950731'], ['n07734744', 'n07715103'], ['n07614500', 'n07615774'], ['n02395406', 'n02410509'], ['n03447447', 'n03662601'], ['n02123045', 'n02123394', 'n02124075'], ['n02666196', 

# Generate dataset

In [ ]:
import torch 

def generate_data_label(mydata, split='train'):
    if split == 'train':
        data_tuple = mydata.train_loader.dataset
    elif split == 'test':
        data_tuple = mydata.test_loader.dataset
    
    data = []
    label_GT = []
    label_comp = []
    num_data = len(data_tuple)
    for i in range(num_data):
        data.append(data_tuple[i][0])
        label_GT.append(data_tuple[i][1])
        label_comp.append(data_tuple[i][2])
    data = torch.stack(data, dim=0) # 4500,3,32,32
    # data = data.reshape(num_data, -1)
    # print(data.shape) # 4500,3072

    ### labels
    labels = []
    for i in range(num_data):
        if label_GT[i] != label_comp[i]:
            tmp = [label_comp[i], '']
        else:
            tmp = [label_comp[i], label_GT[i]]
            # tmp = ['rest', train_label_GT[i]]
        labels.append(tmp)
    return data, labels


In [ ]:
train_data, train_labels = generate_data_label(mydata, split='train')
test_data, test_labels = generate_data_label(mydata, split='test')

In [ ]:
train_data.shape

torch.Size([90000, 3, 224, 224])

In [ ]:
from hiclass import LocalClassifierPerNode
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

train_data_a = train_data.reshape(len(train_data), -1)
train_data_a=train_data_a.numpy()

X_train = train_data_a[-4000:]
Y_train = train_labels[-4000:]

X_test = test_data
# Use random forest classifiers for every node
# rf = RandomForestClassifier()
rf = LogisticRegression()
classifier = LocalClassifierPerNode(local_classifier=rf)

# Train local classifier per node
classifier.fit(X_train, Y_train)

# Predict
predictions = classifier.predict(X_test)
print(predictions)

/home/cxl173430/anaconda3/envs/HENN/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/cxl173430/anaconda3/envs/HENN/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/lin

[['100' '']
 ['107' '']
 ['103' '']
 ...
 ['101' '']
 ['100' '']
 ['108' '']]


In [ ]:
def calculate_metrics(predictions, labels, num_singles, R):
    correct_vague = 0.0
    correct_nonvague = 0.0
    vague_total = 0
    nonvague_total = 0
    num_corr = 0
    for i in range(len(predictions)):
        # singleton acc
        if predictions[i][1]: # singleton prediction
            if int(predictions[i][1]) == labels[i][1]:
                num_corr += 1

        # composite 
        if predictions[i][0]: # composite prediction
            predicted_set = set(R[int(predictions[i][0])])
            k = labels[i][0] # ground truth
            ground_truth_set = set(R[k])
            intersect = predicted_set.intersection(ground_truth_set)
            union = predicted_set.union(ground_truth_set)
            rate = len(intersect) / len(union)
            if int(predictions[i][0])>=num_singles:
                correct_vague += rate
                vague_total += 1
            else:
                correct_nonvague += 1
                nonvague_total += 1
    stat_result = [correct_nonvague, correct_vague, nonvague_total,vague_total]

    overall_js = (stat_result[0] + stat_result[1])/(len(predictions)+1e-10)
    avg_js_vague = stat_result[1] / (stat_result[3]+1e-10)
    acc = num_corr / len(predictions)
    return overall_js, avg_js_vague, acc

In [ ]:
calculate_metrics(predictions, test_labels, mydata.num_classes, mydata.R)

49
33
72
51
71
92
15
14
23
0
71
75
81
69
40
43
97
70
53
70
49
75
29
21
16
39
8
8
70
20
61
41
93
56
73
58
11
25
37
63
24
49
73
56
22
41
58
75
17
4
9
57
2
32
71
52
42
69
77
27
15
65
35
43
82
63
66
90
91
32
32
10
77
22
71
78
54
6
29
89
78
33
11
22
18
27
21
13
21
50
75
37
35
26
83
47
95
43
69
76
17
57
59
25
20
27
0
9
71
8
43
57
56
85
10
19
92
33
20
21
50
70
46
11
16
1
74
33
60
64
52
23
4
11
52
37
24
25
39
51
58
58
77
18
45
66
58
20
24
4
36
8
87
10
30
47
54
99
51
83
37
4
83
95
83
32
73
18
40
39
64
22
28
40
98
83
12
24
45
13
94
24
58
63
7
87
78
68
60
6
23
44
93
73
98
49
90
97
2
67
16
81
94
27
76
77
12
18
0
76
79
71
89
57
47
24
65
0
32
36
82
23
24
34
21
11
53
80
44
4
39
91
16
36
68
50
97
58
31
6
42
80
76
89
55
91
70
1
62
99
51
96
83
42
18
66
40
62
78
84
28
89
30
66
18
38
42
92
27
11
86
44
96
12
16
67
43
89
96
12
40
7
86
77
76
31
18
28
19
18
41
42
43
93
15
10
8
37
89
32
67
12
2
91
94
7
71
36
61
62
5
60
45
34
40
68
62
99
66
46
7
10
10
68
34
37
58
48
40
96
14
11
66
64
39
33
94
63
10
89
92
90
65


(0.11264999999999888, 0.040702702702702265, 0.0078)

In [3]:
from backbones import HENN_EfficientNet, HENN_ResNet50, HENN_VGG16

num_singles = mydata.num_classes
num_comps = mydata.num_comp
print(f"Data: Tiny, num of singleton and composite classes: {num_singles, num_comps}")

num_classes_both = num_singles + num_comps

model_HENN = HENN_EfficientNet(num_classes_both, pretrain=True)
device = "cuda:0"
model_HENN = model_HENN.to(device)
base_path_spec_HENN = "/home/cxl173430/data/uncertainty_Related/HENN_Git_VScode/HyperEvidentialNN_Results/Tiny/GDD_20_15_10M_ker357_sweep_entrGDD_0906/SEED42_10M_Ker5_sweep_GDDexp101/lr_1e-05_klLam_0.0_EntrLamDir_0.0_EntrLamGDD_0"
saved_path_HENN = os.path.join(base_path_spec_HENN, "model_uncertainty_gdd.pt")
checkpoint = torch.load(saved_path_HENN, map_location=device)
model_HENN.load_state_dict(checkpoint["model_state_dict_best"])
print(f"\n### HENN Use the model selected from validation set in Epoch {checkpoint['epoch_best']}:")



Data: Tiny, num of singleton and composite classes: (200, 10)
Loaded pretrained weights for efficientnet-b3

### HENN Use the model selected from validation set in Epoch 98:


In [4]:
import torch.nn as nn
class EfficientNetFeat(nn.Module):
    def __init__(self, pretrained):
        super().__init__()
        self.features = nn.Sequential(
            *list(pretrained.network.children())[:-2]
            )

    def forward(self, x):
        out = self.features(x)
        return out


In [5]:
modela = EfficientNetFeat(model_HENN)
modela 

EfficientNetFeat(
  (features): Sequential(
    (0): Conv2dStaticSamePadding(
      3, 40, kernel_size=(3, 3), stride=(2, 2), bias=False
      (static_padding): ZeroPad2d((0, 1, 0, 1))
    )
    (1): BatchNorm2d(40, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
    (2): ModuleList(
      (0): MBConvBlock(
        (_depthwise_conv): Conv2dStaticSamePadding(
          40, 40, kernel_size=(3, 3), stride=[1, 1], groups=40, bias=False
          (static_padding): ZeroPad2d((1, 1, 1, 1))
        )
        (_bn1): BatchNorm2d(40, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
        (_se_reduce): Conv2dStaticSamePadding(
          40, 10, kernel_size=(1, 1), stride=(1, 1)
          (static_padding): Identity()
        )
        (_se_expand): Conv2dStaticSamePadding(
          10, 40, kernel_size=(1, 1), stride=(1, 1)
          (static_padding): Identity()
        )
        (_project_conv): Conv2dStaticSamePadding(
          

In [ ]:
modela(inputs)

NotImplementedError: Module [ModuleList] is missing the required "forward" function

In [ ]:
list(model_HENN.network.children())[:-2]

[Conv2dStaticSamePadding(
   3, 40, kernel_size=(3, 3), stride=(2, 2), bias=False
   (static_padding): ZeroPad2d((0, 1, 0, 1))
 ),
 BatchNorm2d(40, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True),
 ModuleList(
   (0): MBConvBlock(
     (_depthwise_conv): Conv2dStaticSamePadding(
       40, 40, kernel_size=(3, 3), stride=[1, 1], groups=40, bias=False
       (static_padding): ZeroPad2d((1, 1, 1, 1))
     )
     (_bn1): BatchNorm2d(40, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
     (_se_reduce): Conv2dStaticSamePadding(
       40, 10, kernel_size=(1, 1), stride=(1, 1)
       (static_padding): Identity()
     )
     (_se_expand): Conv2dStaticSamePadding(
       10, 40, kernel_size=(1, 1), stride=(1, 1)
       (static_padding): Identity()
     )
     (_project_conv): Conv2dStaticSamePadding(
       40, 24, kernel_size=(1, 1), stride=(1, 1), bias=False
       (static_padding): Identity()
     )
     (_bn2): BatchNorm2d(2

In [ ]:
model_HENN_feat = EfficientNetFeat(model_HENN)
model_HENN_feat.eval()
with torch.no_grad():
    features = model_HENN_feat(train_data[:10].to(device))

NotImplementedError: Module [ModuleList] is missing the required "forward" function

In [ ]:
model_HENN_feat.features()

torch.nn.modules.container.Sequential

In [ ]:
tmp = train_data[:10].to(device)
tmp.shape

torch.Size([10, 3, 224, 224])

In [ ]:
model_HENN_feat.features(tmp)

NotImplementedError: Module [ModuleList] is missing the required "forward" function

In [ ]:
model_HENN_feat

EfficientNetFeat(
  (features): Sequential(
    (0): Conv2dStaticSamePadding(
      3, 40, kernel_size=(3, 3), stride=(2, 2), bias=False
      (static_padding): ZeroPad2d((0, 1, 0, 1))
    )
    (1): BatchNorm2d(40, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
    (2): ModuleList(
      (0): MBConvBlock(
        (_depthwise_conv): Conv2dStaticSamePadding(
          40, 40, kernel_size=(3, 3), stride=[1, 1], groups=40, bias=False
          (static_padding): ZeroPad2d((1, 1, 1, 1))
        )
        (_bn1): BatchNorm2d(40, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
        (_se_reduce): Conv2dStaticSamePadding(
          40, 10, kernel_size=(1, 1), stride=(1, 1)
          (static_padding): Identity()
        )
        (_se_expand): Conv2dStaticSamePadding(
          10, 40, kernel_size=(1, 1), stride=(1, 1)
          (static_padding): Identity()
        )
        (_project_conv): Conv2dStaticSamePadding(
          

In [ ]:
len(list(model_HENN_feat.children()))

1

In [6]:
with torch.no_grad():
    for batch_idx, (inputs, targets_GT, labels) in enumerate(mydata.train_loader):
        inputs = inputs.to(device, non_blocking=True)
        labels = labels.to(device, non_blocking=True)
        targets_GT = targets_GT.to(device, non_blocking=True)
        break

In [7]:
inputs.shape

torch.Size([64, 3, 224, 224])

In [ ]:
model_HENN_feat(inputs)

NotImplementedError: Module [ModuleList] is missing the required "forward" function

In [ ]:
list(model_HENN.network.children())[0]

Conv2dStaticSamePadding(
  3, 40, kernel_size=(3, 3), stride=(2, 2), bias=False
  (static_padding): ZeroPad2d((0, 1, 0, 1))
)

In [ ]:
featt = nn.Sequential(
    *list(model_HENN.network.children())[:-3]
)
featt

Sequential(
  (0): Conv2dStaticSamePadding(
    3, 40, kernel_size=(3, 3), stride=(2, 2), bias=False
    (static_padding): ZeroPad2d((0, 1, 0, 1))
  )
  (1): BatchNorm2d(40, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
  (2): ModuleList(
    (0): MBConvBlock(
      (_depthwise_conv): Conv2dStaticSamePadding(
        40, 40, kernel_size=(3, 3), stride=[1, 1], groups=40, bias=False
        (static_padding): ZeroPad2d((1, 1, 1, 1))
      )
      (_bn1): BatchNorm2d(40, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
      (_se_reduce): Conv2dStaticSamePadding(
        40, 10, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_se_expand): Conv2dStaticSamePadding(
        10, 40, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_project_conv): Conv2dStaticSamePadding(
        40, 24, kernel_size=(1, 1), stride=(1, 1), bias=False
        (static_paddin

In [ ]:
featt(inputs)

NotImplementedError: Module [ModuleList] is missing the required "forward" function

In [8]:
tmp = model_HENN.network.extract_features(inputs)
tmp.shape

torch.Size([64, 1536, 7, 7])

In [7]:
1536*49

75264

In [8]:
32*32*3

3072

In [9]:
model_HENN.network

EfficientNet(
  (_conv_stem): Conv2dStaticSamePadding(
    3, 40, kernel_size=(3, 3), stride=(2, 2), bias=False
    (static_padding): ZeroPad2d((0, 1, 0, 1))
  )
  (_bn0): BatchNorm2d(40, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
  (_blocks): ModuleList(
    (0): MBConvBlock(
      (_depthwise_conv): Conv2dStaticSamePadding(
        40, 40, kernel_size=(3, 3), stride=[1, 1], groups=40, bias=False
        (static_padding): ZeroPad2d((1, 1, 1, 1))
      )
      (_bn1): BatchNorm2d(40, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
      (_se_reduce): Conv2dStaticSamePadding(
        40, 10, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_se_expand): Conv2dStaticSamePadding(
        10, 40, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_project_conv): Conv2dStaticSamePadding(
        40, 24, kernel_size=(1, 1), stride=(1, 1), bias=False
  

In [10]:
m = nn.AdaptiveAvgPool2d(1)
output = m(tmp)
output.shape


torch.Size([64, 1536, 1, 1])

In [12]:
aa= output.reshape(64, -1)
aa.shape

torch.Size([64, 1536])